In [150]:
import os
import sqlite3
import sys

import pandas as pd
import yaml

In [151]:
# add module path to the python path
# module_path = os.path.abspath(os.path.join('../src'))
# sys.path.insert(0, module_path)

# add project root to python path
notebook_dir = os.path.dirname(os.path.abspath('__file__'))
project_root = os.path.join(notebook_dir, '..')
sys.path.insert(0, project_root)

from src import util

In [152]:
with open('../config.yaml') as file:
    config = yaml.safe_load(file)
config

{'raw_data': {'sample': '../data/raw/Sample-Superstore.csv'},
 'processed_data': {'database_sqlite': '../data/processed/vs_group_order-db.db',
  'database_access': '../data/processed/vs_group_order-db.accdb',
  'sample_parquet': '../data/processed/Sample-Superstore_processed.parquet',
  'sample_csv': '../data/processed/Sample-Superstore_processed.csv'},
 'queries_path': '../src/queries'}

# Constants

In [153]:
DECIMALS_ROUND = 4

# Import

In [154]:
# detect file encoding
# temporarily needed: does not have to run each time
# encoding, confidence = util.detect_encoding(config['raw_data']['sample'])

# print('encoding: ', encoding)
# print('confidence: ', confidence)

In [155]:
df = pd.read_csv(config['raw_data']['sample'], encoding='Windows-1252')

In [156]:
df.head()

,Row ID,Order ID,Order Date,Ship Date,Ship Mode,Customer ID,Customer Name,Segment,Country,City,...,Postal Code,Region,Product ID,Category,Sub-Category,Product Name,Sales,Quantity,Discount,Profit
0,1,CA-2016-152156,11/8/2016,11/11/2016,Second Class,CG-12520,Claire Gute,Consumer,United States,Henderson,...,42420,South,FUR-BO-10001798,Furniture,Bookcases,Bush Somerset Collection Bookcase,261.9600,2,0.00,41.9136
1,2,CA-2016-152156,11/8/2016,11/11/2016,Second Class,CG-12520,Claire Gute,Consumer,United States,Henderson,...,42420,South,FUR-CH-10000454,Furniture,Chairs,"Hon Deluxe Fabric Upholstered Stacking Chairs,...",731.9400,3,0.00,219.5820
2,3,CA-2016-138688,6/12/2016,6/16/2016,Second Class,DV-13045,Darrin Van Huff,Corporate,United States,Los Angeles,...,90036,West,OFF-LA-10000240,Office Supplies,Labels,Self-Adhesive Address Labels for Typewriters b...,14.6200,2,0.00,6.8714
3,4,US-2015-108966,10/11/2015,10/18/2015,Standard Class,SO-20335,Sean O'Donnell,Consumer,United States,Fort Lauderdale,...,33311,South,FUR-TA-10000577,Furniture,Tables,Bretford CR4500 Series Slim Rectangular Table,957.5775,5,0.45,-383.0310
4,5,US-2015-108966,10/11/2015,10/18/2015,Standard Class,SO-20335,Sean O'Donnell,Consumer,United States,Fort Lauderdale,...,33311,South,OFF-ST-10000760,Office Supplies,Storage,Eldon Fold 'N Roll Cart System,22.3680,2,0.20,2.5164


# Normalize

In [157]:
df.columns = df.columns.str.replace(' ', '').str.replace('-', '')

In [158]:
df = df.drop(columns='RowID')

In [159]:
#df = df.drop_duplicates()

In [160]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9994 entries, 0 to 9993
Data columns (total 20 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   OrderID       9994 non-null   object 
 1   OrderDate     9994 non-null   object 
 2   ShipDate      9994 non-null   object 
 3   ShipMode      9994 non-null   object 
 4   CustomerID    9994 non-null   object 
 5   CustomerName  9994 non-null   object 
 6   Segment       9994 non-null   object 
 7   Country       9994 non-null   object 
 8   City          9994 non-null   object 
 9   State         9994 non-null   object 
 10  PostalCode    9994 non-null   int64  
 11  Region        9994 non-null   object 
 12  ProductID     9994 non-null   object 
 13  Category      9994 non-null   object 
 14  SubCategory   9994 non-null   object 
 15  ProductName   9994 non-null   object 
 16  Sales         9994 non-null   float64
 17  Quantity      9994 non-null   int64  
 18  Discount      9994 non-null 

# Set Data Types

In [161]:
df.columns

Index(['OrderID', 'OrderDate', 'ShipDate', 'ShipMode', 'CustomerID',
       'CustomerName', 'Segment', 'Country', 'City', 'State', 'PostalCode',
       'Region', 'ProductID', 'Category', 'SubCategory', 'ProductName',
       'Sales', 'Quantity', 'Discount', 'Profit'],
      dtype='object')

In [162]:
df['OrderID']       = df['OrderID'].astype(pd.StringDtype())
df['OrderDate']     = pd.to_datetime(df['OrderDate']).dt.floor('D')
df['ShipDate']      = pd.to_datetime(df['ShipDate']).dt.floor('D')
df['ShipMode']      = df['ShipMode'].astype('category')
df['CustomerID']    = df['CustomerID'].astype(pd.StringDtype())
df['CustomerName']  = df['CustomerName'].astype(pd.StringDtype())
df['Segment']       = df['Segment'].astype('category')
df['Country']       = df['Country'].astype('category')
df['City']          = df['City'].astype('category')
df['State']         = df['State'].astype('category')
df['Region']        = df['Region'].astype('category')
df['ProductID']     = df['ProductID'].astype(pd.StringDtype())
df['Category']      = df['Category'].astype('category')
df['SubCategory']   = df['SubCategory'].astype('category')
df['ProductName']   = df['ProductName'].astype(pd.StringDtype())
df['Discount']      = df['Discount'].round(decimals=4)
df['Sales']         = df['Sales'].round(decimals=4)
df['Profit']        = df['Profit'].round(decimals=4)

In [163]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9994 entries, 0 to 9993
Data columns (total 20 columns):
 #   Column        Non-Null Count  Dtype         
---  ------        --------------  -----         
 0   OrderID       9994 non-null   string        
 1   OrderDate     9994 non-null   datetime64[ns]
 2   ShipDate      9994 non-null   datetime64[ns]
 3   ShipMode      9994 non-null   category      
 4   CustomerID    9994 non-null   string        
 5   CustomerName  9994 non-null   string        
 6   Segment       9994 non-null   category      
 7   Country       9994 non-null   category      
 8   City          9994 non-null   category      
 9   State         9994 non-null   category      
 10  PostalCode    9994 non-null   int64         
 11  Region        9994 non-null   category      
 12  ProductID     9994 non-null   string        
 13  Category      9994 non-null   category      
 14  SubCategory   9994 non-null   category      
 15  ProductName   9994 non-null   string  

# Making sense of the data

## Meaning of Order ID

In [164]:
df.OrderID.value_counts()

OrderID
CA-2017-100111    14
CA-2017-157987    12
CA-2016-165330    11
US-2016-108504    11
US-2015-126977    10
                  ..
CA-2014-110422     1
CA-2016-125794     1
CA-2017-163566     1
US-2015-151435     1
CA-2016-130225     1
Name: count, Length: 5009, dtype: Int64

In [165]:
df[df.OrderID == 'CA-2017-100111']

,OrderID,OrderDate,ShipDate,ShipMode,CustomerID,CustomerName,Segment,Country,City,State,PostalCode,Region,ProductID,Category,SubCategory,ProductName,Sales,Quantity,Discount,Profit
6090,CA-2017-100111,2017-09-20,2017-09-26,Standard Class,SV-20365,Seth Vernon,Consumer,United States,New York City,New York,10035,East,FUR-CH-10003846,Furniture,Chairs,Hon Valutask Swivel Chairs,272.646,3,0.1,18.1764
6091,CA-2017-100111,2017-09-20,2017-09-26,Standard Class,SV-20365,Seth Vernon,Consumer,United States,New York City,New York,10035,East,TEC-AC-10002647,Technology,Accessories,Logitech Wireless Boombox Speaker - portable -...,212.800,2,0.0,95.7600
6092,CA-2017-100111,2017-09-20,2017-09-26,Standard Class,SV-20365,Seth Vernon,Consumer,United States,New York City,New York,10035,East,OFF-PA-10000807,Office Supplies,Paper,"TOPS ""Important Message"" Pads, Canary, 4-1/4 x...",38.520,9,0.0,18.1044
6093,CA-2017-100111,2017-09-20,2017-09-26,Standard Class,SV-20365,Seth Vernon,Consumer,United States,New York City,New York,10035,East,TEC-AC-10001465,Technology,Accessories,SanDisk Cruzer 64 GB USB Flash Drive,72.640,2,0.0,21.7920
6094,CA-2017-100111,2017-09-20,2017-09-26,Standard Class,SV-20365,Seth Vernon,Consumer,United States,New York City,New York,10035,East,OFF-ST-10000615,Office Supplies,Storage,"SimpliFile Personal File, Black Granite, 15w x...",45.400,4,0.0,12.7120
6095,CA-2017-100111,2017-09-20,2017-09-26,Standard Class,SV-20365,Seth Vernon,Consumer,United States,New York City,New York,10035,East,OFF-PA-10002713,Office Supplies,Paper,"Adams Phone Message Book, 200 Message Capacity...",13.760,2,0.0,6.3296
6096,CA-2017-100111,2017-09-20,2017-09-26,Standard Class,SV-20365,Seth Vernon,Consumer,United States,New York City,New York,10035,East,FUR-CH-10003061,Furniture,Chairs,"Global Leather Task Chair, Black",80.991,1,0.1,8.0991
6097,CA-2017-100111,2017-09-20,2017-09-26,Standard Class,SV-20365,Seth Vernon,Consumer,United States,New York City,New York,10035,East,OFF-BI-10000343,Office Supplies,Binders,"Pressboard Covers with Storage Hooks, 9 1/2"" x...",11.784,3,0.2,3.9771
6098,CA-2017-100111,2017-09-20,2017-09-26,Standard Class,SV-20365,Seth Vernon,Consumer,United States,New York City,New York,10035,East,OFF-FA-10000304,Office Supplies,Fasteners,Advantus Push Pins,4.360,2,0.0,1.7876
6099,CA-2017-100111,2017-09-20,2017-09-26,Standard Class,SV-20365,Seth Vernon,Consumer,United States,New York City,New York,10035,East,FUR-CH-10004086,Furniture,Chairs,Hon 4070 Series Pagoda Armless Upholstered Sta...,2888.127,11,0.1,609.7157


In [166]:
df[df.OrderID == 'CA-2015-131338']

,OrderID,OrderDate,ShipDate,ShipMode,CustomerID,CustomerName,Segment,Country,City,State,PostalCode,Region,ProductID,Category,SubCategory,ProductName,Sales,Quantity,Discount,Profit
1579,CA-2015-131338,2015-08-09,2015-08-12,First Class,NP-18325,Naresj Patel,Consumer,United States,New York City,New York,10024,East,TEC-PH-10003012,Technology,Phones,Nortel Meridian M3904 Professional Digital phone,307.980,2,0.0,89.3142
1580,CA-2015-131338,2015-08-09,2015-08-12,First Class,NP-18325,Naresj Patel,Consumer,United States,New York City,New York,10024,East,FUR-TA-10002607,Furniture,Tables,KI Conference Tables,382.806,9,0.4,-153.1224
1581,CA-2015-131338,2015-08-09,2015-08-12,First Class,NP-18325,Naresj Patel,Consumer,United States,New York City,New York,10024,East,OFF-ST-10000642,Office Supplies,Storage,"Tennsco Lockers, Gray",41.960,2,0.0,2.9372
1582,CA-2015-131338,2015-08-09,2015-08-12,First Class,NP-18325,Naresj Patel,Consumer,United States,New York City,New York,10024,East,OFF-BI-10000545,Office Supplies,Binders,GBC Ibimaster 500 Manual ProClick Binding System,1217.568,2,0.2,456.5880
1583,CA-2015-131338,2015-08-09,2015-08-12,First Class,NP-18325,Naresj Patel,Consumer,United States,New York City,New York,10024,East,FUR-FU-10002157,Furniture,Furnishings,Artistic Insta-Plaque,47.040,3,0.0,18.3456
1584,CA-2015-131338,2015-08-09,2015-08-12,First Class,NP-18325,Naresj Patel,Consumer,United States,New York City,New York,10024,East,FUR-FU-10001706,Furniture,Furnishings,Longer-Life Soft White Bulbs,6.160,2,0.0,2.9568
1585,CA-2015-131338,2015-08-09,2015-08-12,First Class,NP-18325,Naresj Patel,Consumer,United States,New York City,New York,10024,East,TEC-PH-10000984,Technology,Phones,Panasonic KX-TG9471B,979.950,5,0.0,274.3860
1586,CA-2015-131338,2015-08-09,2015-08-12,First Class,NP-18325,Naresj Patel,Consumer,United States,New York City,New York,10024,East,OFF-PA-10001357,Office Supplies,Paper,Xerox 1886,143.700,3,0.0,68.9760
1587,CA-2015-131338,2015-08-09,2015-08-12,First Class,NP-18325,Naresj Patel,Consumer,United States,New York City,New York,10024,East,OFF-FA-10000992,Office Supplies,Fasteners,"Acco Clips to Go Binder Clips, 24 Clips in Two...",10.650,3,0.0,5.0055
1588,CA-2015-131338,2015-08-09,2015-08-12,First Class,NP-18325,Naresj Patel,Consumer,United States,New York City,New York,10024,East,TEC-AC-10002600,Technology,Accessories,Belkin QODE FastFit Bluetooth Keyboard,247.800,4,0.0,34.6920


* OrderID is not unique
* A customer places an order with several products:
  * For each product in an order an additional row is added.
  * Order Date and Shipment Data is typically the same.
  * An Order could however be split into multiple Shipments. 
* OrderID is more an OrderReference.

In [173]:
df = df.rename(columns={'OrderID' : 'OrderReference'})

## Sales and Profit

In [174]:
# What is the meaning of sales? 
df[df.ProductID == 'FUR-TA-10002607']

,OrderReference,OrderDate,ShipDate,ShipMode,CustomerID,CustomerName,Segment,Country,City,State,PostalCode,Region,ProductID,Category,SubCategory,ProductName,Sales,Quantity,Discount,Profit
241,CA-2016-157749,2016-06-04,2016-06-09,Second Class,KL-16645,Ken Lonsdale,Consumer,United States,Chicago,Illinois,60610,Central,FUR-TA-10002607,Furniture,Tables,KI Conference Tables,177.225,5,0.5,-120.5130
746,CA-2014-124429,2014-05-27,2014-05-27,Same Day,MH-17785,Maya Herman,Corporate,United States,San Diego,California,92105,West,FUR-TA-10002607,Furniture,Tables,KI Conference Tables,567.120,10,0.2,-28.3560
1580,CA-2015-131338,2015-08-09,2015-08-12,First Class,NP-18325,Naresj Patel,Consumer,United States,New York City,New York,10024,East,FUR-TA-10002607,Furniture,Tables,KI Conference Tables,382.806,9,0.4,-153.1224
1814,CA-2015-131597,2015-09-14,2015-09-18,Standard Class,SP-20620,Stefania Perrino,Corporate,United States,Los Angeles,California,90045,West,FUR-TA-10002607,Furniture,Tables,KI Conference Tables,170.136,3,0.2,-8.5068
4294,CA-2017-101581,2017-10-22,2017-10-27,Standard Class,DW-13195,David Wiener,Corporate,United States,Redmond,Oregon,97756,West,FUR-TA-10002607,Furniture,Tables,KI Conference Tables,177.225,5,0.5,-120.5130
5373,CA-2015-118738,2015-10-24,2015-10-30,Standard Class,AG-10495,Andrew Gjertsen,Corporate,United States,Houston,Texas,77041,Central,FUR-TA-10002607,Furniture,Tables,KI Conference Tables,347.361,7,0.3,-69.4722
6799,CA-2016-109827,2016-12-25,2017-01-01,Standard Class,LW-16825,Laurel Workman,Corporate,United States,Phoenix,Arizona,85023,West,FUR-TA-10002607,Furniture,Tables,KI Conference Tables,35.445,1,0.5,-24.1026
9031,CA-2017-128041,2017-09-01,2017-09-01,Same Day,RW-19540,Rick Wilson,Corporate,United States,Seattle,Washington,98103,West,FUR-TA-10002607,Furniture,Tables,KI Conference Tables,283.560,4,0.0,45.3696


In [175]:
283.56/4 * 0.5

35.445

* **Sales**: Total Price of order after discount = (1 - Discount) * Price * Quantity
* **Profit**: Total Profit of order; Profit = (ProfitRate * Price - Discount * Price) * Quantity

# Check Data Integrity

In [176]:
# duplicate rows
print('duplicates', df.duplicated().sum())

duplicates 1


In [177]:
# missing values
df.isna().sum()

OrderReference    0
OrderDate         0
ShipDate          0
ShipMode          0
CustomerID        0
CustomerName      0
Segment           0
Country           0
City              0
State             0
PostalCode        0
Region            0
ProductID         0
Category          0
SubCategory       0
ProductName       0
Sales             0
Quantity          0
Discount          0
Profit            0
dtype: int64

no missing values

# Deal with the duplicates in order stem data
order stem data: non-numeric data of an order

In [178]:
# sum up rows where order details are equal, i.e. 'OrderReference', 'CustomerID', 'ProductID', 'OrderDate', 'Discount', 'ShipmentID'
order_stem_data = [
    'OrderReference', 'OrderDate', 'ShipDate', 'ShipMode', 'CustomerID',
    'CustomerName', 'Segment', 'Country', 'City', 'State', 'PostalCode',
    'Region', 'ProductID', 'Category', 'SubCategory', 'ProductName', 'Discount'
    ]

mask_dups = df.duplicated(subset=order_stem_data, keep=False)
print('duplicates', mask_dups.sum())
df[mask_dups]

duplicates 16


,OrderReference,OrderDate,ShipDate,ShipMode,CustomerID,CustomerName,Segment,Country,City,State,PostalCode,Region,ProductID,Category,SubCategory,ProductName,Sales,Quantity,Discount,Profit
350,CA-2016-129714,2016-09-01,2016-09-03,First Class,AB-10060,Adam Bellavance,Home Office,United States,New York City,New York,10009,East,OFF-PA-10001970,Office Supplies,Paper,Xerox 1881,24.560,2,0.0,11.5432
352,CA-2016-129714,2016-09-01,2016-09-03,First Class,AB-10060,Adam Bellavance,Home Office,United States,New York City,New York,10009,East,OFF-PA-10001970,Office Supplies,Paper,Xerox 1881,49.120,4,0.0,23.0864
430,US-2016-123750,2016-04-15,2016-04-21,Standard Class,RB-19795,Ross Baird,Home Office,United States,Gastonia,North Carolina,28052,South,TEC-AC-10004659,Technology,Accessories,Imation Secure+ Hardware Encrypted USB 2.0 Fla...,408.744,7,0.2,76.6395
431,US-2016-123750,2016-04-15,2016-04-21,Standard Class,RB-19795,Ross Baird,Home Office,United States,Gastonia,North Carolina,28052,South,TEC-AC-10004659,Technology,Accessories,Imation Secure+ Hardware Encrypted USB 2.0 Fla...,291.960,5,0.2,54.7425
1300,CA-2016-137043,2016-12-23,2016-12-25,Second Class,LC-17140,Logan Currie,Consumer,United States,Springfield,Virginia,22153,South,FUR-FU-10003664,Furniture,Furnishings,"Electrix Architect's Clamp-On Swing Arm Lamp, ...",572.760,6,0.0,166.1004
1301,CA-2016-137043,2016-12-23,2016-12-25,Second Class,LC-17140,Logan Currie,Consumer,United States,Springfield,Virginia,22153,South,FUR-FU-10003664,Furniture,Furnishings,"Electrix Architect's Clamp-On Swing Arm Lamp, ...",286.380,3,0.0,83.0502
3183,CA-2017-152912,2017-11-09,2017-11-12,Second Class,BM-11650,Brian Moss,Corporate,United States,Columbia,Maryland,21044,East,OFF-ST-10003208,Office Supplies,Storage,Adjustable Depth Letter/Legal Cart,1633.140,9,0.0,473.6106
3184,CA-2017-152912,2017-11-09,2017-11-12,Second Class,BM-11650,Brian Moss,Corporate,United States,Columbia,Maryland,21044,East,OFF-ST-10003208,Office Supplies,Storage,Adjustable Depth Letter/Legal Cart,544.380,3,0.0,157.8702
3405,US-2014-150119,2014-04-23,2014-04-27,Standard Class,LB-16795,Laurel Beltran,Home Office,United States,Columbus,Ohio,43229,East,FUR-CH-10002965,Furniture,Chairs,Global Leather Highback Executive Chair with P...,281.372,2,0.3,-12.0588
3406,US-2014-150119,2014-04-23,2014-04-27,Standard Class,LB-16795,Laurel Beltran,Home Office,United States,Columbus,Ohio,43229,East,FUR-CH-10002965,Furniture,Chairs,Global Leather Highback Executive Chair with P...,281.372,2,0.3,-12.0588


* Some orders are split up into two rows and can be summed up into one row.
* Possible reason: The order has been extended afterwards on the same day.

In [179]:
# sum up
df = df.groupby(by=order_stem_data, observed=True).agg({
    'Sales':'sum', 
    'Quantity':'sum', 
    'Profit':'sum'}
    ).reset_index()

In [180]:
# check again for duplicates
df.duplicated().sum()

np.int64(0)

No duplicates remain after this cleaning step.

# Price Calculation

In [181]:
df['PriceSell']    = (df.Sales/df.Quantity/(1-df.Discount)).round(decimals=DECIMALS_ROUND)
df['PriceBuy']     = ((df.Sales-df.Profit)/df.Quantity).round(decimals=DECIMALS_ROUND)

In [182]:
# checks
print((df['PriceSell']<0).any())
print((df['PriceBuy']<0).any())
print((df['PriceBuy']>=df['PriceSell']).any())
df[(df['PriceBuy']>=df['PriceSell'])]

False
False
True


,OrderReference,OrderDate,ShipDate,ShipMode,CustomerID,CustomerName,Segment,Country,City,State,...,ProductID,Category,SubCategory,ProductName,Discount,Sales,Quantity,Profit,PriceSell,PriceBuy
80,CA-2014-103191,2014-09-22,2014-09-27,Standard Class,VG-21805,Vivek Grady,Corporate,United States,Chicago,Illinois,...,OFF-ST-10002574,Office Supplies,Storage,"SAFCO Commercial Wire Shelving, Black",0.2,331.536,3,-82.884,138.14,138.14
132,CA-2014-104283,2014-06-27,2014-07-01,Standard Class,LM-17065,Liz MacKendrick,Consumer,United States,Southaven,Mississippi,...,OFF-ST-10004337,Office Supplies,Storage,"SAFCO Commercial Wire Shelving, 72h",0.0,306.200,5,0.000,61.24,61.24
140,CA-2014-104563,2014-03-07,2014-03-12,Standard Class,CM-12715,Craig Molinari,Corporate,United States,Seattle,Washington,...,OFF-ST-10000934,Office Supplies,Storage,"Contico 72""H Heavy-Duty Storage System",0.0,204.900,5,0.000,40.98,40.98
509,CA-2014-120775,2014-10-03,2014-10-07,Standard Class,RD-19930,Russell D'Ascenzo,Consumer,United States,Dallas,Texas,...,OFF-FA-10000254,Office Supplies,Fasteners,Sterling Rubber Bands by Alliance,0.2,15.072,4,-3.768,4.71,4.71
594,CA-2014-124646,2014-06-22,2014-06-24,First Class,DV-13465,Dianna Vittorini,Consumer,United States,Minneapolis,Minnesota,...,OFF-ST-10001097,Office Supplies,Storage,Office Impressions Heavy Duty Welded Shelving ...,0.0,501.810,3,0.000,167.27,167.27
854,CA-2014-133424,2014-03-30,2014-04-04,Standard Class,Dl-13600,Dorris liebe,Corporate,United States,Seattle,Washington,...,OFF-ST-10002957,Office Supplies,Storage,Sterilite Show Offs Storage Containers,0.0,15.840,3,0.000,5.28,5.28
909,CA-2014-136336,2014-04-21,2014-04-26,Standard Class,DB-13555,Dorothy Badders,Corporate,United States,Henderson,Kentucky,...,OFF-ST-10002574,Office Supplies,Storage,"SAFCO Commercial Wire Shelving, Black",0.0,828.840,6,0.000,138.14,138.14
917,CA-2014-136567,2014-12-20,2014-12-21,First Class,PS-19045,Penelope Sewall,Home Office,United States,Harrisonburg,Virginia,...,OFF-ST-10004337,Office Supplies,Storage,"SAFCO Commercial Wire Shelving, 72h",0.0,122.480,2,0.000,61.24,61.24
1161,CA-2014-146997,2014-01-23,2014-01-27,Standard Class,SG-20605,Speros Goranitis,Consumer,United States,Lafayette,Indiana,...,OFF-FA-10003467,Office Supplies,Fasteners,"Alliance Big Bands Rubber Bands, 12/Pack",0.0,5.940,3,0.000,1.98,1.98
1580,CA-2014-166863,2014-06-20,2014-06-24,Standard Class,SC-20020,Sam Craven,Consumer,United States,Plano,Texas,...,OFF-ST-10004123,Office Supplies,Storage,Safco Industrial Wire Shelving System,0.2,509.488,7,-127.372,90.98,90.98


There are products that have been sold without a margin!

# Build database tables
* Build tables for the database
* Apply cleaning steps

In [183]:
customer_col    = ['CustomerID','CustomerName', 'Segment']
product_col     = ['ProductID', 'Category', 'SubCategory', 'ProductName', 'PriceSell', 'PriceBuy']
address_col     = ['Country', 'Region', 'State', 'City', 'PostalCode']
shipment_col    = ['ShipDate', 'ShipMode']
orders_col      = ['OrderReference', 'CustomerID', 'ProductID', 'OrderDate','Sales', 'Quantity', 'Discount', 'Profit']

In [184]:
# map col names to db col names
name_mapping={
    'CustomerName'  : 'Name',
    'ProductName'   : 'Name'
}

## Customer

In [185]:
# create dataframes
customer = df[customer_col].drop_duplicates().set_index('CustomerID').rename(columns=name_mapping)
customer.head()


,Name,Segment
CustomerID,,
DK-13375,Dennis Kane,Consumer
EB-13705,Ed Braxton,Corporate
NF-18475,Neil Französisch,Home Office
JC-15340,Jasper Cacioppo,Consumer
JM-15655,Jim Mitchum,Corporate


In [186]:
print('Customer ID duplicates',     customer.index.duplicated().sum())
print('Customer',                   customer.duplicated().sum())
print('Customer Name duplicates',   customer.Name.duplicated().sum())

Customer ID duplicates 0
Customer 0
Customer Name duplicates 0


## Product

In [187]:
def get_product_table(df: pd.DataFrame, cols: list, col_name_mapping: dict) -> pd.DataFrame:
    return df[cols].drop_duplicates().rename(columns=col_name_mapping) 

product = get_product_table(df, product_col, name_mapping) #df[product_col].drop_duplicates().rename(columns=name_mapping) # do not set index, since integrity has to be checked first
product.head()

,ProductID,Category,SubCategory,Name,PriceSell,PriceBuy
0,TEC-PH-10002075,Technology,Phones,AT&T EL51110 DECT,125.99,89.4529
1,FUR-TA-10003715,Furniture,Tables,Hon 2111 Invitation Series Corner Table,209.37,196.8078
2,OFF-BI-10001597,Office Supplies,Binders,"Wilson Jones Ledger-Size, Piano-Hinge Binder, ...",40.98,21.3096
3,OFF-PA-10000176,Office Supplies,Paper,Xerox 1887,18.97,9.8644
4,OFF-BI-10000343,Office Supplies,Binders,"Pressboard Covers with Storage Hooks, 9 1/2"" x...",4.91,2.6023


### Price, Name Duplicates

In [188]:
# Are just the product ID mixed up? Are there products with similar names, but different product ID?
print((product.duplicated(subset=['Category', 'SubCategory', 'Name'], keep=False) & product.duplicated(subset=['Name'], keep=False)).sum())
product.loc[product.duplicated(subset=['Name'], keep=False)].sort_values(['Name', 'ProductID'])

60


,ProductID,Category,SubCategory,Name,PriceSell,PriceBuy
1684,OFF-EN-10000461,Office Supplies,Envelopes,"#10- 4 1/8"" x 9 1/2"" Recycled Envelopes",8.74,4.6322
809,OFF-EN-10000781,Office Supplies,Envelopes,"#10- 4 1/8"" x 9 1/2"" Recycled Envelopes",8.74,4.6322
302,OFF-BI-10000829,Office Supplies,Binders,Avery Non-Stick Binders,4.49,2.4246
45,OFF-BI-10004140,Office Supplies,Binders,Avery Non-Stick Binders,4.49,2.4695
2463,OFF-PA-10000249,Office Supplies,Paper,Easy-staple paper,12.28,6.5084
934,OFF-PA-10000349,Office Supplies,Paper,Easy-staple paper,4.98,2.6394
312,OFF-PA-10000474,Office Supplies,Paper,Easy-staple paper,35.44,18.7832
4269,OFF-PA-10000565,Office Supplies,Paper,Easy-staple paper,55.98,28.5498
1167,OFF-PA-10001685,Office Supplies,Paper,Easy-staple paper,10.14,5.1714
4323,OFF-PA-10002764,Office Supplies,Paper,Easy-staple paper,11.34,5.7834


* There are 60 duplicate product names, whereas there are products with very generic names, like 'Easy-staple paper'.
  * All categories are correct.
* The products differ in pricebuy and/or pricesell, i.e. if the same product is bought at a different price, a new product is created in the database. 
* If there is no difference, then products can be summarized!

* Conclusion: 
  * Keep Name duplicates, and summarize if possible
  * Keep all ProductID duplicates and create a new ProductID (just put the number to a new)
  * Create a new product ID.

In [189]:
# TODO if PriceSell, PriceBuy of equally named items, then summarize to one item (drop_duplicates), re-set items in original dataframe
price_dup_cols = ['Name', 'PriceBuy', 'PriceSell']
price_duplicates = product[product.duplicated(subset=price_dup_cols, keep=False)]
mapping_price_duplicates = pd.merge(price_duplicates.reset_index(), price_duplicates.reset_index(), on=price_dup_cols, how='inner')[['ProductID_x','ProductID_y']]
mapping_price_duplicates = mapping_price_duplicates[mapping_price_duplicates.ProductID_x != mapping_price_duplicates.ProductID_y]
mapping_price_duplicates

,ProductID_x,ProductID_y
1,OFF-EN-10000781,OFF-EN-10000461
3,OFF-EN-10002592,OFF-EN-10003448
4,OFF-EN-10000461,OFF-EN-10000781
7,TEC-MA-10003230,TEC-MA-10001856
8,TEC-MA-10001856,TEC-MA-10003230
10,OFF-EN-10003448,OFF-EN-10002592


In [190]:
# create mapping
mapping_price_duplicates = mapping_price_duplicates.set_index('ProductID_x')['ProductID_y'].to_dict()

# replace in df
df['ProductID'] = df['ProductID'].replace(mapping_price_duplicates)

# replace in product
# product = product[~product.index.isin(mapping_price_duplicates.keys())]
product = get_product_table(df, product_col, name_mapping)

### ProductID Duplicates

In [191]:
# duplicate product IDs
product[product.ProductID.duplicated(keep=False)].sort_index()

,ProductID,Category,SubCategory,Name,PriceSell,PriceBuy
27,OFF-PA-10000357,Office Supplies,Paper,Xerox 1888,55.48,28.8496
29,OFF-ST-10004950,Office Supplies,Storage,Acco Perma 3000 Stacking Storage Drawers,20.98,16.9938
81,FUR-FU-10004091,Furniture,Furnishings,"Eldon 200 Class Desk Accessories, Black",6.28,3.8936
110,OFF-PA-10002195,Office Supplies,Paper,"RSVP Cards & Envelopes, Blank White, 8-1/2"" X ...",5.08,2.6924
163,FUR-FU-10004864,Furniture,Furnishings,"Howard Miller 14-1/2"" Diameter Chrome Round Wa...",63.94,43.4792
...,...,...,...,...,...,...
4436,FUR-FU-10004864,Furniture,Furnishings,Eldon 500 Class Desk Accessories,12.07,8.0869
4854,FUR-CH-10001146,Furniture,Chairs,"Global Task Chair, Black",50.89,45.2921
5463,OFF-PA-10003022,Office Supplies,Paper,Standard Line “While You Were Out” Hardbound T...,21.98,11.8692
8234,TEC-MA-10001148,Technology,Machines,Okidata MB491 Multifunction Printer,499.00,364.2700


There are 64 duplicate product IDs.

In [192]:
# What has changed? Only the names of the products?
mask_dup_subset = product.duplicated(subset=['ProductID', 'Category', 'SubCategory'], keep=False)
print('Duplicates without Name: ', mask_dup_subset.sum())

product[mask_dup_subset].sort_values('ProductID')

Duplicates without Name:  64


,ProductID,Category,SubCategory,Name,PriceSell,PriceBuy
643,FUR-BO-10002213,Furniture,Bookcases,DMI Eclipse Executive Suite Bookcases,500.98,405.7938
2970,FUR-BO-10002213,Furniture,Bookcases,"Sauder Forest Hills Library, Woodland Oak Finish",140.98,129.7016
4854,FUR-CH-10001146,Furniture,Chairs,"Global Task Chair, Black",50.89,45.2921
263,FUR-CH-10001146,Furniture,Chairs,"Global Value Mid-Back Manager's Chair, Gray",60.89,45.6675
2441,FUR-FU-10001473,Furniture,Furnishings,"Eldon Executive Woodline II Desk Accessories, ...",25.13,18.3449
...,...,...,...,...,...,...
2805,TEC-PH-10002200,Technology,Phones,Aastra 6757i CT Wireless VoIP phone,215.44,152.9624
3134,TEC-PH-10002310,Technology,Phones,Plantronics Calisto P620-M USB Wireless Speake...,195.99,143.0727
662,TEC-PH-10002310,Technology,Phones,Panasonic KX T7731-B Digital phone,99.99,72.9927
260,TEC-PH-10004531,Technology,Phones,AT&T CL2909,125.99,88.1930


* All duplicate product IDs differ only in Name.
* We have a fecking error in the system. Where could these errors come from?
  * The name might have changed.
  * The product ID is not a unique identifier or it might have been set wrong.
* Has the product simply been renamed? Scrolling the data:
  * some names are very similar
  * some are not
  * The products have not been renamed; probably they have been mixed up.
  * Possible further check: Check the dates
* Is the product ID really an ID? Or does it refer to a class of products?
  * From the looks it should refer to a specific product.
  * Logically, for the order to be well defined, all information about the product must be given with the information in the above table.
* Solution
  1. Check the number of orders that are affected.
  2. Check if the order IDs have been mixed up.
  3. Calculate the Price and see if that makes a difference.
  4. Drop the duplicates and only keep 32 products
  5. resolve the errors 

In [193]:
# How many entries are affected in the full data set?
mask = df.ProductID.isin(product[mask_dup_subset].ProductID.unique())
print(f'{df[mask].Sales.sum() / df.Sales.sum():.2%} of the total sales volume is affected')

4.24% of the total sales volume is affected


In [194]:
# TODO check if the products already exist and only the product ID was mixed up
pd.merge(product[mask_dup_subset], product, on='Name', how='inner').sort_values('ProductID_x') # self-join

,ProductID_x,Category_x,SubCategory_x,Name,PriceSell_x,PriceBuy_x,ProductID_y,Category_y,SubCategory_y,PriceSell_y,PriceBuy_y
17,FUR-BO-10002213,Furniture,Bookcases,DMI Eclipse Executive Suite Bookcases,500.98,405.7938,FUR-BO-10002213,Furniture,Bookcases,500.98,405.7938
50,FUR-BO-10002213,Furniture,Bookcases,"Sauder Forest Hills Library, Woodland Oak Finish",140.98,129.7016,FUR-BO-10002213,Furniture,Bookcases,140.98,129.7016
60,FUR-CH-10001146,Furniture,Chairs,"Global Task Chair, Black",50.89,45.2921,FUR-CH-10001146,Furniture,Chairs,50.89,45.2921
7,FUR-CH-10001146,Furniture,Chairs,"Global Value Mid-Back Manager's Chair, Gray",60.89,45.6675,FUR-CH-10001146,Furniture,Chairs,60.89,45.6675
44,FUR-FU-10001473,Furniture,Furnishings,"Eldon Executive Woodline II Desk Accessories, ...",25.13,18.3449,FUR-FU-10001473,Furniture,Furnishings,25.13,18.3449
...,...,...,...,...,...,...,...,...,...,...,...
46,TEC-PH-10002200,Technology,Phones,Aastra 6757i CT Wireless VoIP phone,215.44,152.9624,TEC-PH-10002200,Technology,Phones,215.44,152.9624
54,TEC-PH-10002310,Technology,Phones,Plantronics Calisto P620-M USB Wireless Speake...,195.99,143.0727,TEC-PH-10002310,Technology,Phones,195.99,143.0727
19,TEC-PH-10002310,Technology,Phones,Panasonic KX T7731-B Digital phone,99.99,72.9927,TEC-PH-10002310,Technology,Phones,99.99,72.9927
6,TEC-PH-10004531,Technology,Phones,AT&T CL2909,125.99,88.1930,TEC-PH-10004531,Technology,Phones,125.99,88.1930


The dataframe has the same shape, i.e. there are not other products with the name of the duplicated productIDs. The IDs must have been created wrongly by the system!

In [195]:
product.head()

,ProductID,Category,SubCategory,Name,PriceSell,PriceBuy
0,TEC-PH-10002075,Technology,Phones,AT&T EL51110 DECT,125.99,89.4529
1,FUR-TA-10003715,Furniture,Tables,Hon 2111 Invitation Series Corner Table,209.37,196.8078
2,OFF-BI-10001597,Office Supplies,Binders,"Wilson Jones Ledger-Size, Piano-Hinge Binder, ...",40.98,21.3096
3,OFF-PA-10000176,Office Supplies,Paper,Xerox 1887,18.97,9.8644
4,OFF-BI-10000343,Office Supplies,Binders,"Pressboard Covers with Storage Hooks, 9 1/2"" x...",4.91,2.6023


In [196]:
# fix in product and in df
# dismantle product id into 3 parts
productids = pd.DataFrame()
productids[['part1', 'part2', 'part3']] = product.ProductID.str.split('-', expand=True)
productids['ProductID'] = product.ProductID
productids['Name'] = product.Name
productids.sort_values(by=['part1', 'part2', 'part3'])
productids.part3 = productids.part3.astype('int')

# check if part3 is unique
tmp = productids.drop_duplicates(subset='ProductID')
tmp[tmp.part3.duplicated(keep=False)].sort_values(by='part3')

# check if part3 is unique in each category
# tmp.groupby(['part1', 'part2']).apply(lambda g: g.part3.unique())
tmp['part3_duplicated_per_category'] = tmp.groupby(['part1', 'part2'])['part3'].transform(lambda x: x.duplicated(keep=False))
tmp.groupby(['part1', 'part2'])['part3_duplicated_per_category'].any()

# increment the number in 3rd part to the next free spot for each category
base_number = 10000000
productids['part3_reset'] = productids.groupby(['part1', 'part2']).cumcount() + base_number
productids.sort_values(by=['part1', 'part2', 'part3_reset'])
productids['ProductID_reset'] = productids['part1'] + '-' + productids['part2'] + '-' + productids['part3_reset'].astype('str')

# create mapping in order to replace in df and product
mapping_product_id_reset = productids[['Name', 'ProductID', 'ProductID_reset']].set_index(['Name', 'ProductID'])['ProductID_reset'].to_dict()

# replace in df
df = pd.merge(
    df,
    productids[['Name', 'ProductID', 'ProductID_reset']].rename(columns={'Name':'ProductName'}), 
    on=['ProductName', 'ProductID'],
    how='left'
    ).rename(columns={'ProductID':'ProductID_old', 'ProductID_reset':'ProductID'})


C:\Users\olive\AppData\Local\Temp\ipykernel_23516\3691149988.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  tmp['part3_duplicated_per_category'] = tmp.groupby(['part1', 'part2'])['part3'].transform(lambda x: x.duplicated(keep=False))


In [197]:
product.head()

,ProductID,Category,SubCategory,Name,PriceSell,PriceBuy
0,TEC-PH-10002075,Technology,Phones,AT&T EL51110 DECT,125.99,89.4529
1,FUR-TA-10003715,Furniture,Tables,Hon 2111 Invitation Series Corner Table,209.37,196.8078
2,OFF-BI-10001597,Office Supplies,Binders,"Wilson Jones Ledger-Size, Piano-Hinge Binder, ...",40.98,21.3096
3,OFF-PA-10000176,Office Supplies,Paper,Xerox 1887,18.97,9.8644
4,OFF-BI-10000343,Office Supplies,Binders,"Pressboard Covers with Storage Hooks, 9 1/2"" x...",4.91,2.6023


In [198]:
# recreate product
product = get_product_table(df, product_col, name_mapping)
if not product.ProductID.duplicated().any():
    product = product.set_index('ProductID')
else:
    raise Exception('Index is not unique')

In [199]:
product.head()

,Category,SubCategory,Name,PriceSell,PriceBuy
ProductID,,,,,
TEC-PH-10000000,Technology,Phones,AT&T EL51110 DECT,125.99,89.4529
FUR-TA-10000000,Furniture,Tables,Hon 2111 Invitation Series Corner Table,209.37,196.8078
OFF-BI-10000000,Office Supplies,Binders,"Wilson Jones Ledger-Size, Piano-Hinge Binder, ...",40.98,21.3096
OFF-PA-10000000,Office Supplies,Paper,Xerox 1887,18.97,9.8644
OFF-BI-10000001,Office Supplies,Binders,"Pressboard Covers with Storage Hooks, 9 1/2"" x...",4.91,2.6023


### Integrity
* Product ID: cat-subcat-number
* integrity check: check if category and subcategory are ok


In [200]:
# TODO


### Separate product categories

In [201]:
product_all = product.copy()
product_all

,Category,SubCategory,Name,PriceSell,PriceBuy
ProductID,,,,,
TEC-PH-10000000,Technology,Phones,AT&T EL51110 DECT,125.99,89.4529
FUR-TA-10000000,Furniture,Tables,Hon 2111 Invitation Series Corner Table,209.37,196.8078
OFF-BI-10000000,Office Supplies,Binders,"Wilson Jones Ledger-Size, Piano-Hinge Binder, ...",40.98,21.3096
OFF-PA-10000000,Office Supplies,Paper,Xerox 1887,18.97,9.8644
OFF-BI-10000001,Office Supplies,Binders,"Pressboard Covers with Storage Hooks, 9 1/2"" x...",4.91,2.6023
...,...,...,...,...,...
OFF-AR-10000163,Office Supplies,Art,4009 Highlighters,1.34,0.8844
OFF-LA-10000069,Office Supplies,Labels,Avery 5,2.88,1.4688
OFF-EN-10000053,Office Supplies,Envelopes,"Peel & Seel Recycled Catalog Envelopes, Brown",11.58,5.7900


In [202]:
product_categories = product_all[['Category', 'SubCategory']].drop_duplicates().reset_index(drop=True)
product = pd.merge(
    product_all.reset_index(),
    product_categories.reset_index(), 
    on=['Category', 'SubCategory'], how='left'
    ).drop(columns=['Category', 'SubCategory']).rename(columns={'index':'ProductCategoryID'})
product = product.set_index('ProductID')
product.head()

,Name,PriceSell,PriceBuy,ProductCategoryID
ProductID,,,,
TEC-PH-10000000,AT&T EL51110 DECT,125.99,89.4529,0
FUR-TA-10000000,Hon 2111 Invitation Series Corner Table,209.37,196.8078,1
OFF-BI-10000000,"Wilson Jones Ledger-Size, Piano-Hinge Binder, ...",40.98,21.3096,2
OFF-PA-10000000,Xerox 1887,18.97,9.8644,3
OFF-BI-10000001,"Pressboard Covers with Storage Hooks, 9 1/2"" x...",4.91,2.6023,2


## Address

In [203]:
address = df[address_col].drop_duplicates().reset_index(drop=True)#.set_index('ProductID').rename(columns=name_mapping)
address.head()

,Country,Region,State,City,PostalCode
0,United States,East,New York,New York City,10024
1,United States,West,California,San Francisco,94122
2,United States,South,Florida,Jacksonville,32216
3,United States,West,Arizona,Glendale,85301
4,United States,East,New York,New York City,10035


### check data integrity

In [204]:
address.Country.value_counts()
address.Region.value_counts()
address.State.value_counts()
# address.City.value_counts()
# address.PostalCode.value_counts()

State
California              90
Texas                   59
Florida                 38
Illinois                35
Ohio                    24
Michigan                23
New York                21
Washington              19
North Carolina          19
New Jersey              18
Colorado                16
Tennessee               15
Indiana                 15
Massachusetts           14
Arizona                 13
Minnesota               13
Wisconsin               13
Virginia                13
Pennsylvania            11
Georgia                 11
Connecticut             10
Utah                    10
Arkansas                 9
Missouri                 9
Oregon                   9
Iowa                     8
Kentucky                 8
Alabama                  8
Maryland                 7
New Mexico               7
Oklahoma                 7
Idaho                    6
South Carolina           6
Nevada                   5
Montana                  5
Louisiana                5
Kansas                

In [205]:
print('nunique ["City", "PostalCode"]: ', address[['City', 'PostalCode']].nunique())
print('duplicates ["City", "PostalCode"]: ', address[['City', 'PostalCode']].duplicated().sum())

nunique ["City", "PostalCode"]:  City          531
PostalCode    631
dtype: int64
duplicates ["City", "PostalCode"]:  0


In [206]:
address.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 632 entries, 0 to 631
Data columns (total 5 columns):
 #   Column      Non-Null Count  Dtype   
---  ------      --------------  -----   
 0   Country     632 non-null    category
 1   Region      632 non-null    category
 2   State       632 non-null    category
 3   City        632 non-null    category
 4   PostalCode  632 non-null    int64   
dtypes: category(4), int64(1)
memory usage: 30.2 KB


### build secondary address info table

In [207]:
address_secondary = address[['Country', 'Region', 'State']].drop_duplicates()
address_secondary.info()

<class 'pandas.core.frame.DataFrame'>
Index: 49 entries, 0 to 557
Data columns (total 3 columns):
 #   Column   Non-Null Count  Dtype   
---  ------   --------------  -----   
 0   Country  49 non-null     category
 1   Region   49 non-null     category
 2   State    49 non-null     category
dtypes: category(3)
memory usage: 2.3 KB


In [208]:
# separate secondary address info from address
address_all = address
address = pd.merge(address, address_secondary.reset_index(), on=['Country', 'Region', 'State']).drop(columns=['Country', 'Region', 'State']).rename(columns={'index':'AddressSecondaryID'})

## Shipment

In [209]:
shipment_all = df[shipment_col+address_col].drop_duplicates().reset_index(drop=True)

In [210]:
merge_col = ['Country', 'City', 'State', 'PostalCode','Region']
shipment = pd.merge(
    shipment_all, address_all.reset_index(), how='inner', 
    on=merge_col
    ).rename(columns={'index':'AddressID'}).drop(columns=merge_col)

In [211]:
shipment.head()

,ShipDate,ShipMode,AddressID
0,2014-09-13,Standard Class,0
1,2014-07-12,Standard Class,1
2,2014-03-18,Standard Class,2
3,2014-02-03,Standard Class,0
4,2014-04-15,Standard Class,3


In [212]:
shipment.shape

(4973, 3)

## Orders

In [213]:
orders_all = df[orders_col+shipment_col+address_col]

In [214]:
orders_all.head()

,OrderReference,CustomerID,ProductID,OrderDate,Sales,Quantity,Discount,Profit,ShipDate,ShipMode,Country,Region,State,City,PostalCode
0,CA-2014-100006,DK-13375,TEC-PH-10000000,2014-09-07,377.970,3,0.0,109.6113,2014-09-13,Standard Class,United States,East,New York,New York City,10024
1,CA-2014-100090,EB-13705,FUR-TA-10000000,2014-07-08,502.488,3,0.2,-87.9354,2014-07-12,Standard Class,United States,West,California,San Francisco,94122
2,CA-2014-100090,EB-13705,OFF-BI-10000000,2014-07-08,196.704,6,0.2,68.8464,2014-07-12,Standard Class,United States,West,California,San Francisco,94122
3,CA-2014-100293,NF-18475,OFF-PA-10000000,2014-03-14,91.056,6,0.2,31.8696,2014-03-18,Standard Class,United States,South,Florida,Jacksonville,32216
4,CA-2014-100328,JC-15340,OFF-BI-10000001,2014-01-28,3.928,1,0.2,1.3257,2014-02-03,Standard Class,United States,East,New York,New York City,10024


In [215]:
orders_all.duplicated().sum()

np.int64(0)

In [216]:
merge_col = shipment_col+address_col
orders = pd.merge(orders_all, 
         shipment_all.reset_index(), 
         how='left', 
         on=merge_col).drop(columns=merge_col).rename(columns={'index':'ShipmentID'})

In [217]:
orders = orders.rename(columns=name_mapping)

In [218]:
orders.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9986 entries, 0 to 9985
Data columns (total 9 columns):
 #   Column          Non-Null Count  Dtype         
---  ------          --------------  -----         
 0   OrderReference  9986 non-null   string        
 1   CustomerID      9986 non-null   string        
 2   ProductID       9986 non-null   string        
 3   OrderDate       9986 non-null   datetime64[ns]
 4   Sales           9986 non-null   float64       
 5   Quantity        9986 non-null   int64         
 6   Discount        9986 non-null   float64       
 7   Profit          9986 non-null   float64       
 8   ShipmentID      9986 non-null   int64         
dtypes: datetime64[ns](1), float64(3), int64(2), string(3)
memory usage: 702.3 KB


In [219]:
order_prim_key = ['ProductID', 'CustomerID', 'OrderReference', 'ShipmentID', 'OrderReference', 'OrderDate']
prim_key_duplicated = orders.duplicated(subset=order_prim_key, keep=False)
if prim_key_duplicated.any():
    orders[prim_key_duplicated]

# Export

In [220]:
%store df

Stored 'df' (DataFrame)


## to database

In [224]:
from pathlib import Path
db_path = Path(config['processed_data']['database_sqlite'])
query_path = Path(config['queries_path'])
print(query_path.exists() and db_path.exists())

True


In [225]:
def init_db(conn):
    # reset db
    with open(query_path.joinpath('drop_tables.sql'), 'r') as f:
        conn.executescript(f.read())

    # setup tables
    with open(query_path.joinpath('create_tables.sql'), 'r') as f:
        conn.executescript(f.read())
    # with open(query_path.joinpath('create_views.sql'), 'r') as f:
    #     conn.executescript(f.read())

    # write data
    customer            .to_sql('customer', conn, if_exists='append', index_label='ID')
    product             .to_sql('product', conn, if_exists='append', index_label='ID')
    product_categories  .to_sql('productcategories', conn, if_exists='append', index_label='ID')
    address             .to_sql('address', conn, if_exists='append', index_label='ID')
    address_secondary   .to_sql('addresssecondary', conn, if_exists='append', index_label='ID')
    shipment            .to_sql('shipment', conn, if_exists='append', index_label='ID')
    orders              .to_sql('orders', conn, if_exists='append', index=False)

In [229]:
# sqlite db
with sqlite3.connect(db_path) as conn:
    init_db(conn)

# Deliver data for Analytics System (tableau dashboard)

In [227]:
import sys
print(sys.version)
print(sys.maxsize > 2**32) # True for 64-bit, False for 32-bit

3.13.5 | packaged by conda-forge | (main, Jun 16 2025, 08:20:19) [MSC v.1943 64 bit (AMD64)]
True


In [230]:
# access db for tableau
# import pyodbc

# with pyodbc.connect(config['processed_data']['database_access']) as conn:
#     init_db(conn)

In [233]:
# export full data
with sqlite3.connect(db_path) as conn:
    with open(query_path.joinpath('select_data_for_tableau_dashboard.sql'), 'r') as f:
        # cur = conn.executescript(f.read())
        dashboard_data = pd.read_sql_query(f.read(), conn)
        dashboard_data.to_parquet(config['processed_data']['sample_parquet'])
        dashboard_data.to_csv(config['processed_data']['sample_csv'], index=False, encoding='utf-8')

In [234]:
dashboard_data.head()

,OrderDate,ProductID,CustomerID,ShipmentID,OrderReference,Quantity,Discount,Sales,Profit,CustomerName,...,ProductName,Category,SubCategory,ShipDate,ShipMode,PostalCode,City,Country,Region,State
0,2014-09-07 00:00:00,TEC-PH-10000000,DK-13375,0,CA-2014-100006,3,0.0,377.970,109.6113,Dennis Kane,...,AT&T EL51110 DECT,Technology,Phones,2014-09-13 00:00:00,Standard Class,10024,New York City,United States,East,New York
1,2014-10-19 00:00:00,TEC-PH-10000001,EH-14125,10,CA-2014-100867,6,0.2,321.552,20.0970,Eugene Hildebrand,...,RCA Visys Integrated PBX 8-Line Router,Technology,Phones,2014-10-24 00:00:00,Standard Class,90712,Lakewood,United States,West,California
2,2014-11-21 00:00:00,TEC-PH-10000001,JK-15325,4328,US-2014-168501,5,0.2,267.960,16.7475,Jason Klamczynski,...,RCA Visys Integrated PBX 8-Line Router,Technology,Phones,2014-11-27 00:00:00,Standard Class,75220,Dallas,United States,Central,Texas
3,2015-06-16 00:00:00,TEC-PH-10000001,LC-16885,4459,US-2015-163825,2,0.0,133.980,33.4950,Lena Creighton,...,RCA Visys Integrated PBX 8-Line Router,Technology,Phones,2015-06-19 00:00:00,First Class,10009,New York City,United States,East,New York
4,2017-01-20 00:00:00,TEC-PH-10000001,TH-21100,4011,CA-2017-161809,3,0.2,160.776,10.0485,Thea Hendricks,...,RCA Visys Integrated PBX 8-Line Router,Technology,Phones,2017-01-26 00:00:00,Standard Class,90045,Los Angeles,United States,West,California


# Remarks: Insights into Business Processes from Data Cleaning
* A new product is created for each buying process. The product may have already been sold with a different product reference / ID.
  * This has to be taken into account in the product analysis: group by Category and Name, average the prices.
* 